## Import

In [37]:
import warnings
warnings.filterwarnings("ignore")

# Imports
import time
from IPython.display import display, Javascript
import itertools
import duckdb
import pandas as pd
import numpy as np
import math

# Plot
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression

# Data process
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PowerTransformer

# Train
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

# Models
import umap
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score

# Install sqlite as a extension of duckdb
#duckdb.install_extension('sqlite')

In [20]:
# create a connection to a file called 'file.db'
con = duckdb.connect("../data/train/diamonds_train.db")

# Query to extract data from database
query_full = """
SELECT
    --tra.index_id,
    cut.cut,
    col.color,
    cla.clarity,
    tra.price,
    cit.city,
    tra.carat,
    dim.depth,
    dim.table,
    dim.x,
    dim.y,
    dim.z
FROM diamonds_properties AS pro
JOIN diamonds_cut AS cut ON pro.cut_id = cut.cut_id
JOIN diamonds_color AS col ON pro.color_id = col.color_id
JOIN diamonds_clarity AS cla ON pro.clarity_id = cla.clarity_id
JOIN diamonds_transactional as tra ON pro.index_id = tra.index_id
JOIN diamonds_city AS cit ON tra.city_id = cit.city_id
JOIN diamonds_dimensions AS dim ON pro.index_id = dim.index_id
"""

diamond_train_df = con.execute(query_full).df()
diamond_train_df.head()
df = diamond_train_df

In [9]:
diamond_test_df = pd.read_csv("../data/test/diamonds_test.csv")
diamond_test_df = diamond_test_df.drop(['id'], axis=1)
diamond_test_df.head()

,carat,cut,color,clarity,depth,table,x,y,z,city
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [21]:
cut_categories = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
df['cut'] = pd.Categorical(df['cut'], categories= cut_categories, ordered=True)
df['cut']

0          Premium
1        Very Good
2             Fair
3             Good
4            Ideal
           ...    
40450        Ideal
40451        Ideal
40452    Very Good
40453      Premium
40454        Ideal
Name: cut, Length: 40455, dtype: category
Categories (5, object): ['Fair' < 'Good' < 'Very Good' < 'Premium' < 'Ideal']

In [22]:
color_categories = sorted(df['color'].value_counts().index, reverse=True)
df['color'] = pd.Categorical(df['color'], categories= color_categories, ordered=True)
df['color']

0        J
1        H
2        G
3        D
4        G
        ..
40450    I
40451    F
40452    E
40453    F
40454    E
Name: color, Length: 40455, dtype: category
Categories (7, object): ['J' < 'I' < 'H' < 'G' < 'F' < 'E' < 'D']

In [23]:
clarity_categories = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']
df['clarity'] = pd.Categorical(df['clarity'], categories= clarity_categories, ordered=True)
df['clarity']

0        VS2
1        VS2
2        VS1
3        SI1
4        SI1
        ... 
40450    SI1
40451    VS1
40452    SI1
40453    VS1
40454     I1
Name: clarity, Length: 40455, dtype: category
Categories (8, object): ['I1' < 'SI2' < 'SI1' < 'VS2' < 'VS1' < 'VVS2' < 'VVS1' < 'IF']

In [40]:
valores_unicos = df['city'].unique()
df['city'] = pd.Categorical(df['city'], categories= valores_unicos, ordered=True)
df['city']

0                Dubai
1             Kimberly
2            Las Vegas
3             Kimberly
4                Dubai
             ...      
40450       Luxembourg
40451         Kimberly
40452         Kimberly
40453        Amsterdam
40454    New York City
Name: city, Length: 40455, dtype: category
Categories (13, object): ['Dubai' < 'Kimberly' < 'Las Vegas' < 'Tel Aviv' ... 'Amsterdam' < 'Luxembourg' < 'London' < 'New York City']

In [43]:
cat_cols = ['cut','color','clarity', 'city']
cat_orders = [cut_categories, color_categories, clarity_categories, valores_unicos]
encoder = OrdinalEncoder(categories=cat_orders)
cats_encoded = pd.DataFrame(encoder.fit_transform(df[cat_cols]), columns = ['cut_encoded','color_encoded','clarity_encoded','city_encoded'])
cats_encoded

,cut_encoded,color_encoded,clarity_encoded,city_encoded
0,3.0,0.0,3.0,0.0
1,2.0,2.0,3.0,1.0
2,0.0,3.0,4.0,2.0
3,1.0,6.0,2.0,1.0
4,4.0,3.0,2.0,0.0
...,...,...,...,...
40450,4.0,1.0,2.0,10.0
40451,4.0,4.0,4.0,1.0
40452,2.0,5.0,2.0,1.0
40453,3.0,4.0,4.0,9.0


In [44]:
df_encoded = df.drop(columns=['cut','color','clarity']).copy()

df_encoded['cut_encoded'] = df['cut'].cat.codes
df_encoded['color_encoded'] = df['color'].cat.codes
df_encoded['clarity_encoded'] = df['clarity'].cat.codes
df_encoded['city'] = df['city'].cat.codes

In [45]:
df_encoded.head()

,price,city,carat,depth,table,x,y,z,volume,cut_encoded,color_encoded,clarity_encoded
0,4268,0,1.21,62.4,58.0,6.83,6.79,4.25,26438.6304,3,0,3
1,505,1,0.32,63.0,57.0,4.35,4.38,2.75,17384.2200,2,2,3
2,2686,2,0.71,NaN,55.0,5.62,5.53,3.65,23725.0825,0,3,4
3,738,1,0.41,63.8,56.0,4.68,4.72,3.00,19212.4768,1,6,2
4,4882,0,1.02,60.5,59.0,6.55,6.51,3.95,23828.2275,4,3,2


In [46]:
#df.rename(columns={'x': 'length', 'y': 'width', 'depth': 'depth%', 'z':'depth','table':'table_width'}, inplace=True)
df['volume'] = df['depth'] * df['y'] * df['depth']
df

,price,city,carat,depth,table,x,y,z,volume,cut,color,clarity
0,4268,Dubai,1.21,62.4,58.0,6.83,6.79,4.25,26438.6304,Premium,J,VS2
1,505,Kimberly,0.32,63.0,57.0,4.35,4.38,2.75,17384.2200,Very Good,H,VS2
2,2686,Las Vegas,0.71,NaN,55.0,5.62,5.53,3.65,NaN,Fair,G,VS1
3,738,Kimberly,0.41,63.8,56.0,4.68,4.72,3.00,19212.4768,Good,D,SI1
4,4882,Dubai,1.02,60.5,59.0,6.55,6.51,3.95,23828.2275,Ideal,G,SI1
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,6300,Luxembourg,1.15,61.8,55.0,6.73,6.76,4.17,25818.0624,Ideal,I,SI1
40451,1800,Kimberly,0.53,61.4,57.0,5.18,5.20,3.19,19603.7920,Ideal,F,VS1
40452,1641,Kimberly,0.58,63.1,56.0,5.36,5.32,3.37,21182.1652,Very Good,E,SI1
40453,2368,Amsterdam,0.66,60.8,58.0,5.67,5.64,3.44,20849.0496,Premium,F,VS1


In [28]:
df_encoded['volume'] = df['volume']
df_encoded

,price,city,carat,depth,table,x,y,z,cut_encoded,color_encoded,clarity_encoded,volume
0,4268,Dubai,1.21,62.4,58.0,6.83,6.79,4.25,3,0,3,26438.6304
1,505,Kimberly,0.32,63.0,57.0,4.35,4.38,2.75,2,2,3,17384.2200
2,2686,Las Vegas,0.71,65.5,55.0,5.62,5.53,3.65,0,3,4,23725.0825
3,738,Kimberly,0.41,63.8,56.0,4.68,4.72,3.00,1,6,2,19212.4768
4,4882,Dubai,1.02,60.5,59.0,6.55,6.51,3.95,4,3,2,23828.2275
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,6300,Luxembourg,1.15,61.8,55.0,6.73,6.76,4.17,4,1,2,25818.0624
40451,1800,Kimberly,0.53,61.4,57.0,5.18,5.20,3.19,4,4,4,19603.7920
40452,1641,Kimberly,0.58,63.1,56.0,5.36,5.32,3.37,2,5,2,21182.1652
40453,2368,Amsterdam,0.66,60.8,58.0,5.67,5.64,3.44,3,4,4,20849.0496


In [30]:
clean_df = df_encoded.copy()

outliers_cols = ['y','z','table','depth','volume']

# setting values that above or lower than the whiskers in the box plot to NaNs
for col in outliers_cols:

    data = clean_df[col]
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)

    IQR = Q3 - Q1

    min = Q1 - (1.5 * IQR)
    max = Q3 + (1.5 * IQR)
 
    outliers = ( (data < min) | (data > max) )

    clean_df.loc[outliers, col] = np.nan
    
clean_df.isna().sum()

price                 0
city                  0
carat                 0
depth              1899
table               447
x                     0
y                    22
z                    37
cut_encoded           0
color_encoded         0
clarity_encoded       0
volume               41
dtype: int64

In [31]:
df_all = clean_df.copy()

df_all[['cut','color','clarity']] = df[['cut','color','clarity']] 

df = df_all.drop(columns=['cut_encoded','color_encoded','clarity_encoded'])

In [35]:
Skewed_Cols = clean_df[['carat','volume']].columns
trans_df = clean_df.copy()
for col in Skewed_Cols:
    trans_df[col] = np.log(1 + trans_df[col])

In [38]:
df_ploy = trans_df.drop(columns=['price']).copy()

poly = PolynomialFeatures(2)

ploy_data = poly.fit_transform(df_ploy)

df_ploy = pd.DataFrame(ploy_data, columns=poly.get_feature_names_out())

df_ploy

ValueError: could not convert string to float: 'Dubai'